In [1]:
import pandas_plink
from pandas_plink import read_plink1_bin
from os import path
import dask.array as da
import numpy as np

from lmdec.decomp.iter_methods import PowerMethod

In [61]:
data_loc = '../../1000G_Pruned'
data_dir = path.abspath(path.join(data_loc, '1KG_pruned_forPCA'))
bed_file = path.abspath(path.join(data_loc, '1KG_pruned_forPCA.bed'))
bim_file = path.abspath(path.join(data_loc, '1KG_pruned_forPCA.bim'))
fam_file = path.abspath(path.join(data_loc, '1KG_pruned_forPCA.fam'))

In [62]:
type(fam_file)


str

In [63]:
import pathlib

In [64]:
isinstance(pathlib.Path(fam_file), pathlib.Path)

True

In [65]:
bed_file

'/Users/tnonet/Documents/1000G_Pruned/1KG_pruned_forPCA.bed'

In [67]:
G = pandas_plink.read_plink1_bin(bed_file, bim_file, fam_file)


Mapping files: 100%|██████████| 3/3 [00:00<00:00,  5.20it/s]


In [70]:
G.data

,Array,Chunk
Bytes,3.37 GB,8.39 MB
Shape,"(2504, 168047)","(1024, 1024)"
Count,2475 Tasks,495 Chunks
Type,float64,numpy.ndarray


In [24]:
PM = PowerMethod(k=10, tol=1e-4, scoring_method='rmse', factor='n')

In [25]:
G_ma = da.ma.masked_invalid(G)

In [26]:
U, S, V = PM.svd(G_ma)

In [27]:
PM._factor

'n'

In [28]:
PM.num_iter

2

In [29]:
PM.history['acc']

{'q-vals': [],
 'rmse': [0.001986638853030779, 4.0059166301446854e-05],
 'v-subspace': []}

In [2]:
from lmdec.array.core.matrix_ops import svd_to_trunc_svd
from lmdec.array.core.metrics import *


In [2]:


PM = PowerMethod(k=100)
array = np.random.rand(50, 50)
# # with pytest.raises(ValueError):
_, _, _ = PM.svd(array)


Exception: Cannot split more than unity. p = 11.0

In [10]:
list(x not in scoring_method for x in ['q-vals', 'rmse', 'v-subspace'])

[False, True, True]

In [21]:
n, p = 10, 7
a = np.random.random((10, 7))
aat = a.dot(a.T)

U, S, _ = np.linalg.svd(aat, full_matrices=False)
V, _, _ = np.linalg.svd(a.T.dot(U), full_matrices=False)
np.testing.assert_almost_equal(rmse_k(a, U, S), 0)
print(a.T.shape, V.T.shape, S.shape)
np.testing.assert_almost_equal(rmse_k(a.T, V, S[:p]), 0)

(7, 10) (7, 7) (10,)


In [2]:
array = np.random.rand(100, 70)
k = 10
tol_groups = [[1e-1, 1e-1, 1e-1],
              [1e-16, 1e-4, 1-16],
              [1e-6, 1e-16, 1e-16],
              [1e-16, 1e-16, 1e-6]]

for tols in tol_groups:
    PM = PowerMethod(k=k, tol=tols, scoring_method=['q-vals', 'rmse', 'v-subspace'])

In [ ]:
20.051998**.5

In [4]:
x = slice(0,10)

In [6]:
x.stop

10

In [7]:
A = np.random.randn(10,10)

In [10]:
A[[*[1,2],3], :]

array([[-1.06054703e+00, -1.65028779e+00, -2.26402629e-01,
        -8.82735743e-01,  1.09451491e+00, -6.42056228e-01,
        -9.45006790e-03,  1.97625072e+00,  1.34739187e-01,
        -9.01808764e-01],
       [-6.98789207e-01, -1.20281770e+00, -4.39291441e-01,
         9.01872605e-01, -3.68410905e-01,  1.08914915e-02,
         7.55117164e-01,  1.64518340e+00,  4.48785141e-01,
        -5.11339048e-01],
       [ 2.74543044e-01,  1.73186317e+00, -6.79403622e-01,
         8.87979234e-01, -9.10432977e-01, -1.97236811e-03,
        -1.19858515e+00,  1.63982908e+00,  9.54948589e-01,
         4.53823112e-01]])

In [19]:
from lmdec.array.core.random import array_constant_partition, array_geometric_partition, cumulative_partition
import numpy as np

In [32]:
n, p = 1000, 1000
a = np.random.randn(n, p)
f = .1
parts = array_constant_partition(a.shape, f=f, min_size=1, axis=1)
cum_parts = cumulative_partition(parts)

for i, cum_part in enumerate(cum_parts):
    a_actual = a[:, 0:int((i + 1) * p * f)]
    a_part_stack = np.hstack([a[:, p] for p in parts[:i+1]])
    a_cum = a[:, cum_part]

    #np.testing.assert_array_equal(a_actual, a_part_stack)
    #np.testing.assert_array_equal(a_actual, a_cum)


In [33]:
a_actual.shape

(1000, 100)

In [24]:
a_part_stack.shape

(100, 1000)

In [16]:
a = range(10)

In [18]:
list(a[:0])

[]

In [14]:
np.vstack?

Signature: np.vstack(tup)
Docstring:
Stack arrays in sequence vertically (row wise).

This is equivalent to concatenation along the first axis after 1-D arrays
of shape `(N,)` have been reshaped to `(1,N)`. Rebuilds arrays divided by
`vsplit`.

This function makes most sense for arrays with up to 3 dimensions. For
instance, for pixel-data with a height (first axis), width (second axis),
and r/g/b channels (third axis). The functions `concatenate`, `stack` and
`block` provide more general stacking and concatenation operations.

Parameters
----------
tup : sequence of ndarrays
    The arrays must have the same shape along all but the first axis.
    1-D arrays must have the same length.

Returns
-------
stacked : ndarray
    The array formed by stacking the given arrays, will be at least 2-D.

See Also
--------
stack : Join a sequence of arrays along a new axis.
hstack : Stack arrays in sequence horizontally (column wise).
dstack : Stack arrays in sequence depth wise (along third dimensi